# Lab 06-1 Softmax Classification Eager
* Softmax를 사용해 분류를 진행

### 기본 Library 선언 및 Tensorflow 버전 확인

In [1]:
import tensorflow as tf
import numpy as np

tf.random.set_seed(777)
print(tf.__version__)

2.3.0


In [2]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

# convert into numpy and float format
# np.array vs np.asarray => copy vs (데이터 형태가 같을 경우 : view / 다를 경우 : copy)
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [3]:
nb_classes = 3 # class 갯수

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [4]:
# Weight and bias setting
W = tf.Variable(tf.random.normal((4, nb_classes)), name ='weight')
b = tf.Variable(tf.random.normal((nb_classes, )), name ='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.7706481 ,  0.37335402, -0.05576323],
       [ 0.00358377, -0.5898363 ,  1.5702795 ],
       [ 0.2460895 , -0.09918973,  1.4418385 ],
       [ 0.3200988 ,  0.526784  , -0.7703731 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.3080608 , -0.13253094,  0.5513761 ], dtype=float32)>


### Softmax function

> - $k$ : data 갯수
- $$ softmax(x_i) =  \frac{e^{x_i}}{\sum _{j = 0}^{k} e^{x_j}} \quad (i = 0,1,...k)$$

In [12]:
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[1.36571955e-02 7.90162385e-03 9.78441238e-01]
 [3.92597765e-02 1.70347411e-02 9.43705440e-01]
 [3.80385250e-01 1.67723164e-01 4.51891571e-01]
 [3.23390484e-01 5.90759404e-02 6.17533624e-01]
 [3.62997412e-06 6.20727292e-08 9.99996305e-01]
 [2.62520202e-02 1.07279615e-02 9.63019967e-01]
 [1.56525111e-05 4.21802781e-07 9.99983907e-01]
 [2.94076904e-06 3.81133276e-08 9.99997020e-01]], shape=(8, 3), dtype=float32)


In [13]:
# Softmax one hot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[0.9302204  0.06200533 0.00777428]], shape=(1, 3), dtype=float32)


### Cost function

> - Cross-Entropy Loss
- $C$ : Class 갯수
- $f()$ : Softmax function
- $$ CE =  - \frac {1}{k} \sum _{i}^{C} t_i log(f(s)_i) $$

In [20]:
def cost_func(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis = 1)
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

print(cost_func(x_data, y_data))

tf.Tensor(6.07932, shape=(), dtype=float32)


In [22]:
def grad_func(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_func(X, Y)
        grads = tape.gradient(loss, variables)
        
        return grads
    
print(grad_func(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.06914607, -0.6509784 ,  0.5818323 ],
       [-1.5221257 , -1.214863  ,  2.7369885 ],
       [-1.2473828 , -1.7611003 ,  3.008483  ],
       [-1.2014606 , -1.8659233 ,  3.0673838 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.15212913, -0.34219202,  0.4943211 ], dtype=float32)>]


In [25]:
def fit(X, Y, epochs = 2001, verbose = 100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
    
    for i in range(epochs):
        grads = grad_func(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_func(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 1.732344
Loss at epoch 100: 0.631795
Loss at epoch 200: 0.556792
Loss at epoch 300: 0.504043
Loss at epoch 400: 0.460375
Loss at epoch 500: 0.421357
Loss at epoch 600: 0.384780
Loss at epoch 700: 0.349172
Loss at epoch 800: 0.313413
Loss at epoch 900: 0.277111
Loss at epoch 1000: 0.246472
Loss at epoch 1100: 0.232681
Loss at epoch 1200: 0.221539
Loss at epoch 1300: 0.211379
Loss at epoch 1400: 0.202075
Loss at epoch 1500: 0.193524
Loss at epoch 1600: 0.185638
Loss at epoch 1700: 0.178345
Loss at epoch 1800: 0.171580
Loss at epoch 1900: 0.165290
Loss at epoch 2000: 0.159427


### Prediction Check

In [31]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype = np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1))

tf.Tensor([[0.00112728 0.08148751 0.9173852 ]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [32]:
b = hypothesis(x_data)

print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1))

tf.Tensor(
[[2.1849892e-06 1.2296562e-03 9.9876815e-01]
 [1.1272823e-03 8.1487469e-02 9.1738522e-01]
 [2.2046049e-07 1.6407260e-01 8.3592719e-01]
 [6.3632988e-06 8.5055929e-01 1.4943434e-01]
 [2.6134700e-01 7.2661775e-01 1.2035291e-02]
 [1.3774875e-01 8.6222386e-01 2.7332080e-05]
 [7.4253356e-01 2.5742957e-01 3.6859663e-05]
 [9.2206264e-01 7.7936828e-02 5.9709657e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


### Convert as Class

In [39]:
class softmax_classifier(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifier, self).__init__()
        self.W = tf.Variable(tf.random.normal((4, nb_classes)), name = 'weight')
        self.b = tf.Variable(tf.random.normal((nb_classes, )), name = 'bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_func(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_mean(Y * tf.math.log(logits), axis = 1))
        return cost
    
    def grad_func(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_func(X, Y)
            grads = tape.gradient(cost, self.variables)
            return grads
        
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimzer = tf.keras.optimizers.SGD(learning_rate=0.5)
        
        for i in range(epochs):
            grads = self.grad_func(X, Y)
            optimzer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.cost_func(X, Y).numpy()))
                
                
model = softmax_classifier(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 1.914325
Loss at epoch 500: 0.174164
Loss at epoch 1000: 0.113783
Loss at epoch 1500: 0.035754
Loss at epoch 2000: 0.029387
